In [1]:
import pandas as pd
import re

In [2]:
lst_header = ['num_exp', 'dia', 'hora', 'calle', 'num', 'distrito', 'tipo_accidente', 'meteo', 'vehiculo', 'persona', 'edad', 'sexo', 'lesividad' ]
bici = pd.read_csv('bicicletas/AccidentesBicicletas_2019.csv', encoding='latin_1',sep = ';', names=lst_header)

In [3]:
bici.head()

,num_exp,dia,hora,calle,num,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad
0,Nº EXPEDIENTE,FECHA,HORA,CALLE,NÚMERO,DISTRITO,TIPO ACCIDENTE,ESTADO METEREOLÓGICO,TIPO VEHÍCULO,TIPO PERSONA,RANGO DE EDAD,SEXO,LESIVIDAD*
1,2019S000659,01/01/2019,14:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ,-,SALAMANCA,Alcance,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,01
2,2019S000036,02/01/2019,20:45,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES,-,HORTALEZA,Colisión fronto-lateral,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,03
3,2019S000133,03/01/2019,14:30,CALL. FELIPE ALVAREZ,10,VILLA DE VALLECAS,Alcance,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,07
4,2019S000132,03/01/2019,12:45,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA,-,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,07


In [4]:
bici.dtypes

num_exp           object
dia               object
hora              object
calle             object
num               object
distrito          object
tipo_accidente    object
meteo             object
vehiculo          object
persona           object
edad              object
sexo              object
lesividad         object
dtype: object

In [5]:
bici.drop(0, inplace=True)
bici.reset_index(drop=True, inplace=True)

In [6]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    bici['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

bici = create_date(bici)

In [7]:
def create_dir(df):
    df.num.apply(lambda x: x.strip())
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

bici = create_dir(bici)

In [8]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Alcance,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,01,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colisión fronto-lateral,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,03,2019-01-02 20:45:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Alcance,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,07,2019-01-03 14:30:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,07,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 21 A 24 AÑOS,Hombre,14,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [9]:
bici.tipo_accidente.isna().sum()

0

In [10]:
def change_accident(values):
    dict_accidente = {
        r'Caída':'Caida',
        r'Colisión':'Colision',
        r'Alcance':'Colision',
        r'Atropello':'Imprudencia',
        r'Choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values):
            return v
    return 'Otro'

bici.tipo_accidente = bici.tipo_accidente.apply(lambda x: change_accident(x))

In [11]:
bici.tipo_accidente.value_counts()

Colision       388
Caida          230
Imprudencia    141
Otro            20
Name: tipo_accidente, dtype: int64

In [12]:
def change_injury(values):
    dict_injury = {
        '01':'Moderada','02':'Moderada',
        '03':'Grave',
        '04':'Fallecido',
        '05':'Leve','06':'Leve','07':'Leve','14':'Leve'}
    return dict_injury[values]

bici.lesividad = bici.lesividad.apply(lambda x: change_injury(str(x)))    

KeyError: 'nan'

In [18]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Colision,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,None,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colision,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,None,2019-01-02 20:45:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Colision,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,None,2019-01-03 14:30:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,None,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Bicicleta,Conductor,DE 21 A 24 AÑOS,Hombre,None,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [14]:
bici[bici.tipo_accidente == 'Otro'].lesividad.value_counts()

Series([], Name: lesividad, dtype: int64)

In [22]:
bici[bici.lesividad.isna()].tipo_accidente.value_counts()

Alcance                         20
Colisión fronto-lateral         18
Colisión lateral                12
Caída                           11
Atropello a persona              9
Colisión frontal                 3
Choque contra obstáculo fijo     3
Otro                             3
Colisión múltiple                1
Name: tipo_accidente, dtype: int64

In [11]:
bici.persona.value_counts()

Conductor    707
Peatón        68
Pasajero       4
Name: persona, dtype: int64

In [12]:
bici.sexo.value_counts()

Hombre    574
Mujer     192
Name: sexo, dtype: int64

In [ ]:
bici.drop(['sexo', 'vehiculo'], axis=1, inplace = True)

In [ ]:
# encoding='cp850'
# encoding='cp1250'

# **
# encoding='latin_1'
# encoding='iso8859_15'